In [2]:
from llama_generation import LLAMA_Generation

import os

os.environ['MASTER_ADDR'] = 'localhost'
os.environ['MASTER_PORT'] = '12357'
os.environ['RANK'] = '0'
os.environ['WORLD_SIZE'] = '1'

agent = LLAMA_Generation(
        ckpt_dir='llama-2-7b-chat/',
        tokenizer_path='tokenizer.model',
        max_seq_len=512 * 6,
        max_batch_size=8
)

> initializing model parallel with size 1
> initializing ddp with size 1
> initializing pipeline with size 1


/data/shaoshus/miniconda3/envs/roboranking/lib/python3.9/site-packages/torch/__init__.py:696: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:451.)
  _C._set_default_tensor_type(t)


Loaded in 11.50 seconds


# judge professorship

In [3]:
agent.instructions_text

"You are tasked with analyzing the information provided to determineif the individual in question holds the position of a professor.based on the information available.Please provide your response in the following format: 'YES', 'NO', or 'NOT SURE'.You can answer 'YES' only when that text mentions that the individual is a professor.You can answer 'YES' only when that text has direct confirmation of professorshipResearcher is not a direct confirmation of professorship."

In [5]:
content_text =\
    "Changhwan Kim (Member, IEEE) received the B.S. degree in mechanical engineering and the M.S. degree in machine design engineering from Hanyang University, Seoul, South Korea, in 1993 and 1995, respectively, and the Ph.D. degree in mechanical engineering from The University of Iowa, Iowa City, IA, USA, in 2002. From 2002 to 2004, he was a Research Associate with the Robotics and Automation Laboratory, University of Notre Dame, Notre Dame, IN, USA. Since 2004, he has been working at the Korea Institute of Science and Technology (KIST), Seoul. His research interests include task and motion planning for robot manipulation and social robots.(Based on document published on 26 December 2022)."


# content_text = "N/A"
# content_text = "Yan Zhuang (Senior Member, IEEE) is not a professor"

import time
start = time.time()

response, original_answer = agent.judge_professorship(content_text, verbose=False)


# remove space in string
# response = response.replace(" ", "")
print(response)
print(original_answer)
print(time.time() - start)

NO
 Based on the information provided, I would answer 'NO' because the text does not directly confirm that Changhwan Kim is a professor. While it mentions that he received a Ph.D. degree in mechanical engineering from The University of Iowa in 2002, it does not specify his current academic position or title. Therefore, I cannot confirm with certainty that he holds the position of a professor.
3.5884029865264893


In [3]:
import json
with open(f'auther_database_v5_5_2_extended_no_score.json', 'r') as file:
    author_database = json.load(file)


import time

min_papers = 10 
count = 0

for pid, author in author_database.items():
    
    # if 'robor_affiliation' not in author and ["is_professor"] == 'None':
    if 'is_professor' in author:
        if author['is_professor'] == 'None' and author['number_of_ieee_paper'] >10:
        # if author['is_professor'] == 'None' and author['num_papers'] >10:
        # if author['is_professor'] == 'None' and author['number_of_ieee_paper'] >10 and 'ieee_affiliation' not in author:
            
            print()
            print(pid)
            print(author['number_of_ieee_paper'])
            print(author['ieee_affiliation'])
            print(author['ieee_biography'])

            if author['ieee_biography'] == 'N/A':
                response = 'NO'
                original_answer = 'N/A'
            else:
                response,original_answer = agent.judge_professorship(author['ieee_biography'], verbose=False)

            start = time.time()
            # remove space in string
            # response = response.replace(" ", "")
            print(response)
            print(original_answer)
            print(time.time() - start)

            count += 1

            if count > 10:
                break

print(count)
# print(f" > {min_papers} papers \t {count}")


# with open(f'auther_database_v5_5_1_extended_no_score.json', 'w') as file:
#     json.dump(author_database, file, indent=4)


07/7628
11
['Munich Institute of Robotics and Machine Intelligence (MIRMI), Technische Universität München (TUM), Germany']
N/A
NO
N/A
1.049041748046875e-05

123/6833
11
['WIRobotics, Yongin, South Korea']
Biography
Younbaek Lee received the M.S. degree in mechanical engineering from Korea Advanced Institute of Science and Technology, Daejeon, South Korea, in 2003.,He is currently a research staff member of the Samsung Advanced Institute of Technology, South Korea. His research interests include electromechanical design, medical/rehabilitation robotics, humanoids, manipulators and physical human-machine interaction.(Based on document published on 14 June 2019).
NOT SURE
 Based on the information provided, I would answer "NOT SURE" whether the individual in question holds the position of a professor. The document does not directly confirm that the person is a professor, and the title of the position is not mentioned. The individual's current role is mentioned as a research staff member

# Find affiliation

In [6]:
# read csv to string

# with open('country-info3_3.csv', 'r') as file:
#     author_database = json.load(file)

import csv

file_path = 'country-info3_3.csv'  # Replace this with your file path

with open(file_path, newline='') as csvfile:
    reader = csv.reader(csvfile)
    next(reader, None)  # Skip the header row
    first_column_values = [row[0] for row in reader if row]  # Check if row is not empty
    csv_string = '\n'.join(first_column_values)


instruct_text = \
    "Based on the given information, please provide which of the following institute is the people belong to. " \
    "If the information is not enough to determine the institute, please provide the answer as 'N/A'.\n" + \
csv_string

agent.instructions_text = instruct_text

content_text = "'Centre for Advanced Robotics Technology Innovation (CARTIN)', 'School of Electrical and Electronic Engineering', 'Nanyang Technological University, Singapore'"


affiliation = agent.judge_affiliation(content_text, verbose=False)

print(affiliation)


AssertionError: 

# delete the agent and empty the GPU memory

In [8]:
del agent
import gc
gc.collect()
# clena the GPU memory
import torch
torch.cuda.empty_cache()